# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Imported the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray <file.path>\nosql-challenge\Scripts\establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'auto_db',
 'config',
 'epa',
 'fruit_db',
 'local',
 'mets',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo["uk_food"]

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection + save as variable
establishments_c = db["establishments"]
establishments_c.find_one()

{'_id': ObjectId('63ef301581e5f0fbe1b6c861'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Create a dictionary for the new restaurant data
new_restaurant = {"BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True}

In [8]:
# Insert the new restaurant into the collection
establishments_c.insert_one(new_restaurant)

In [9]:
# Check that the new restaurant was inserted
list(establishments_c.find({"BusinessName":"Penang Flavours"}))

[{'_id': ObjectId('63ef30290a1732eba2670915'),
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'PostCode': 'SE18 7DY',
  'Phone': '',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
  'RightToReply': '',
  'Distance': 4623.972328074718,
  'NewRatingPending': True}]

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}

pprint((establishments_c.find_one(query,fields)))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63ef301581e5f0fbe1b6c861')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [11]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName":"Penang Flavours"}
update = {"$set": {"BusinessTypeID": 1}}

establishments_c.update_one(query, update)

In [12]:
# Confirm that the new restaurant was updated
fields = {"BusinessName":1, "BusinessTypeID": 1, "BusinessType": 1}
pprint(list(establishments_c.find({"BusinessName":"Penang Flavours"}, fields)))

[{'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('63ef30290a1732eba2670915')}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [13]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
print(establishments_c.count_documents(query))

994


In [14]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments_c.delete_many(query)

In [15]:
# Check if any remaining documents include Dover
print(establishments_c.count_documents(query))

0


In [16]:
# Check that other documents remain with 'find_one'
pprint(establishments_c.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63ef301581e5f0fbe1b6cb48'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# Change the data type from String to Decimal for longitude
query = {}
update = [{'$set': {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]

establishments_c.update_many(query, update)

In [18]:
# Change the data type from String to Decimal for latitude
query = {}
update = [{'$set': {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]

establishments_c.update_many(query, update)

In [19]:
# Check that the coordinates are now numbers - searching by type double -> if it's a string it won't appear
query = {"geocode.longitude": {"$type": "double"}, "geocode.latitude": { "$type": "double"}}
fields = {"geocode.longitude":1,"geocode.latitude":1}
pprint(list(establishments_c.find(query, fields).limit(5)))


[{'_id': ObjectId('63ef301581e5f0fbe1b6cb48'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'_id': ObjectId('63ef301581e5f0fbe1b6cb49'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408}},
 {'_id': ObjectId('63ef301581e5f0fbe1b6cb4a'),
  'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'_id': ObjectId('63ef301581e5f0fbe1b6cb4d'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63ef301581e5f0fbe1b6cb4e'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}]
